In [16]:
import arcpy
import os

def diagnose_and_repair_geometries(input_fc, output_gdb, run_repair=True):
    """
    Diagnose and optionally repair invalid geometries from a feature class.

    Args:
        input_fc (str): Path to input feature class (e.g. .sde or .gdb feature class)
        output_gdb (str): Path to output file geodatabase (will be created if doesn't exist)
        run_repair (bool): Whether to attempt geometry repair using buffer(0)

    Returns:
        dict: Summary of diagnostics and (optional) repairs
    """
    arcpy.env.overwriteOutput = True

    # Create output GDB if it doesn't exist
    if not arcpy.Exists(output_gdb):
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    spatial_ref = arcpy.Describe(input_fc).spatialReference

    # Output feature class names
    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None

    # Create output schemas
    arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                        geometry_type="POLYGON", spatial_reference=spatial_ref)
    if run_repair:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

    invalid_count = 0
    repaired_count = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]


    with arcpy.da.SearchCursor(input_fc, fields) as search_cursor, \
         arcpy.da.InsertCursor(invalid_fc, ["SHAPE@"]) as invalid_writer, \
         (arcpy.da.InsertCursor(repaired_fc, ["SHAPE@"]) if run_repair else None) as repaired_writer:

        while True:
            try:
                row = next(search_cursor)
            except StopIteration:
                break
            except Exception as fetch_err:
                print(f"❌ Failed to read row from cursor: {fetch_err}")
                continue

            try:
                oid, shape, treefarm_id, parcelnumber = row

                if shape is None:
                    print(f"⚠️ Null shape at OID {oid}")
                    continue

                if not shape.isValid:
                    invalid_writer.insertRow((shape,))
                    invalid_count += 1

                    if run_repair:
                        try:
                            repaired_shape = shape.buffer(0)
                            if repaired_shape and repaired_shape.isValid:
                                repaired_writer.insertRow((repaired_shape,))
                                repaired_count += 1
                            else:
                                print(f"⚠️ Repaired still invalid (OID {oid})")
                        except Exception as repair_err:
                            print(f"❌ Repair failed at OID {oid}: {repair_err}")

            except Exception as row_process_err:
                print(f"❌ Failed to process geometry at row: {row_process_err}")


    summary = {
        "🔎 Total Invalid Geometries": invalid_count,
        "🛠️  Total Repaired Geometries": repaired_count if run_repair else "Not run",
        "📍 Invalid Geometries Output": invalid_fc,
        "🧽 Repaired Geometries Output": repaired_fc if run_repair else "Not run"
    }

    print("\n📋 Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    return summary

result = diagnose_and_repair_geometries(
    input_fc = r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
    output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
    run_repair=True
)



❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm

KeyboardInterrupt: 

In [ ]:
import arcpy
import os
import csv

unreadable_count = 0

def diagnose_and_repair_geometries(input_fc, output_gdb, run_repair=True):
    arcpy.env.overwriteOutput = True

    # Ensure output GDB exists
    if not arcpy.Exists(output_gdb):
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    spatial_ref = arcpy.Describe(input_fc).spatialReference

    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None
    log_csv = os.path.join(output_gdb, "GeometryDiagnostics.csv")

    arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                        geometry_type="POLYGON", spatial_reference=spatial_ref)
    if run_repair:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

    invalid_count = 0
    repaired_count = 0
    unreadable_count = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]



    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["OID", "TreeFarmID", "ParcelNumber", "Status", "Details"])

        with arcpy.da.SearchCursor(input_fc, fields) as search_cursor, \
             arcpy.da.InsertCursor(invalid_fc, ["SHAPE@"]) as invalid_writer, \
             (arcpy.da.InsertCursor(repaired_fc, ["SHAPE@"]) if run_repair else None) as repaired_writer:

            while True:
                try:
                    row = next(search_cursor)
                except StopIteration:
                    break
                except Exception as fetch_err:
                    unreadable_count += 1
                    # ❌ Log as unreadable with minimal info
                    logwriter.writerow(["?", "?", "?", "Unreadable", str(fetch_err)])
                    continue

                try:
                    oid, shape, treefarm_id, parcelnumber = row

                    if shape is None:
                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Null Geometry", "Shape is None"])
                        continue

                    if not shape.isValid:
                        invalid_writer.insertRow((shape,))
                        invalid_count += 1
                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Invalid", "Failed isValid()"])

                        if run_repair:
                            try:
                                repaired_shape = shape.buffer(0)
                                if repaired_shape and repaired_shape.isValid:
                                    repaired_writer.insertRow((repaired_shape,))
                                    repaired_count += 1
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repaired", "Buffer(0) succeeded"])
                                else:
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Failed", "Still invalid"])
                            except Exception as repair_err:
                                logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Error", str(repair_err)])
                    else:
                        # Log valid if you want full audit
                        # logwriter.writerow([oid, treefarm_id, parcelnumber, "Valid", ""])
                        pass

                except Exception as row_process_err:
                    unreadable_count += 1
                    logwriter.writerow(["?", "?", "?", "Processing Error", str(row_process_err)])
                    continue





    summary = {
        "🔎 Invalid Geometries": invalid_count,
        "🧽 Repaired Geometries": repaired_count if run_repair else "Not run",
        "🚫 Unreadable Features": unreadable_count,
        "📍 Invalid Features Output": invalid_fc,
        "🚫 Unreadable Features Skipped": unreadable_count,
        "🛠️  Repaired Features Output": repaired_fc if run_repair else "Not run",
        "📝 Diagnostics CSV": log_csv
    }

    print("\n📋 Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    return summary

result = diagnose_and_repair_geometries(
    input_fc = r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
    output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
    run_repair=True
)


In [2]:
import arcpy
import os
import csv
import time
from datetime import datetime

def diagnose_and_repair_geometries(input_fc, output_gdb, run_repair=True):
    """
    Diagnose and repair geometries with comprehensive logging and progress tracking.
    """
    print(f"🚀 Starting geometry validation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Get total record count for progress tracking
    try:
        total_records = int(arcpy.GetCount_management(input_fc).getOutput(0))
        print(f"📊 Total records to process: {total_records:,}")
    except Exception as e:
        print(f"⚠️ Could not get record count: {e}")
        total_records = None

    # Ensure output GDB exists
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    try:
        spatial_ref = arcpy.Describe(input_fc).spatialReference
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")
    except Exception as e:
        print(f"❌ Failed to get spatial reference: {e}")
        return {"error": f"Failed to get spatial reference: {e}"}

    # Output paths
    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None
    log_csv = os.path.join(os.path.dirname(output_gdb), "GeometryDiagnostics.csv")  # Save CSV outside GDB

    # Create output feature classes
    print("🏗️ Creating output feature classes...")
    try:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

        # Add the data fields to invalid geometries FC
        arcpy.AddField_management(invalid_fc, "ORIG_OID", "LONG")
        arcpy.AddField_management(invalid_fc, "treefarm_id", "TEXT", field_length=50)
        arcpy.AddField_management(invalid_fc, "parcelnumber", "TEXT", field_length=50)

        if run_repair:
            arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                                geometry_type="POLYGON", spatial_reference=spatial_ref)
            # Add the data fields to repaired geometries FC
            arcpy.AddField_management(repaired_fc, "ORIG_OID", "LONG")
            arcpy.AddField_management(repaired_fc, "treefarm_id", "TEXT", field_length=50)
            arcpy.AddField_management(repaired_fc, "parcelnumber", "TEXT", field_length=50)

    except Exception as e:
        print(f"❌ Failed to create output feature classes: {e}")
        return {"error": f"Failed to create output feature classes: {e}"}

    # Initialize counters
    invalid_count = 0
    repaired_count = 0
    unreadable_count = 0
    null_geometry_count = 0
    valid_count = 0
    processed_count = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]

    print("📝 Creating diagnostics CSV and starting processing...")

    # Progress tracking variables
    last_report_time = time.time()
    report_interval = 1000  # Report every 1000 records

    try:
        with open(log_csv, mode="w", newline="") as csvfile:
            logwriter = csv.writer(csvfile)
            logwriter.writerow(["OID", "TreeFarmID", "ParcelNumber", "Status", "Details", "Timestamp"])

            # Use regular for loop instead of while True for better performance
            # Add where clause to skip problematic records if needed
            try:
                search_cursor = arcpy.da.SearchCursor(input_fc, fields)
            except Exception as cursor_err:
                print(f"❌ Failed to create cursor: {cursor_err}")
                return {"error": f"Failed to create cursor: {cursor_err}"}

                # Create insert cursors with data fields
                invalid_insert_fields = ["SHAPE@", "ORIG_OID", "treefarm_id", "parcelnumber"]
                repaired_insert_fields = ["SHAPE@", "ORIG_OID", "treefarm_id", "parcelnumber"]

                with arcpy.da.InsertCursor(invalid_fc, invalid_insert_fields) as invalid_writer, \
                     (arcpy.da.InsertCursor(repaired_fc, repaired_insert_fields) if run_repair else None) as repaired_writer:

                    try:
                        for row in search_cursor:
                            try:
                                processed_count += 1

                                # Progress reporting
                                if processed_count % report_interval == 0 or (time.time() - last_report_time) >= 30:
                                    elapsed = time.time() - start_time
                                    if total_records:
                                        progress = (processed_count / total_records) * 100
                                        estimated_total = elapsed * total_records / processed_count
                                        remaining = estimated_total - elapsed
                                        print(f"⏳ Progress: {processed_count:,}/{total_records:,} ({progress:.1f}%) - "
                                              f"Invalid: {invalid_count:,} - Valid: {valid_count:,} - "
                                              f"Elapsed: {elapsed/60:.1f}min - Est. remaining: {remaining/60:.1f}min")
                                    else:
                                        print(f"⏳ Processed: {processed_count:,} - Invalid: {invalid_count:,} - "
                                              f"Valid: {valid_count:,} - Elapsed: {elapsed/60:.1f}min")
                                    last_report_time = time.time()

                                # Process the row
                                oid, shape, treefarm_id, parcelnumber = row
                                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                                # Handle null geometries
                                if shape is None:
                                    null_geometry_count += 1
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Null Geometry", "Shape is None", current_time])
                                    continue

                                # Check geometry validity with enhanced error handling
                                try:
                                    is_valid = shape.isValid
                                except Exception as validity_err:
                                    # Handle geometries that can't even be checked for validity
                                    unreadable_count += 1
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Validity Check Failed", str(validity_err), current_time])
                                    print(f"⚠️ Cannot check validity for OID {oid}: {validity_err}")
                                    continue

                                if not is_valid:
                                    invalid_count += 1

                                    # Try to insert invalid geometry - handle insertion errors
                                    try:
                                        invalid_writer.insertRow([shape, oid, treefarm_id, parcelnumber])
                                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Invalid", "Failed isValid()", current_time])
                                    except Exception as insert_err:
                                        # If we can't even insert the invalid geometry, log it
                                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Insert Failed", f"Cannot insert invalid geometry: {insert_err}", current_time])
                                        print(f"⚠️ Cannot insert invalid geometry for OID {oid}: {insert_err}")

                                    if run_repair:
                                        try:
                                            repaired_shape = shape.buffer(0)
                                            if repaired_shape and repaired_shape.isValid:
                                                repaired_count += 1
                                                try:
                                                    repaired_writer.insertRow([repaired_shape, oid, treefarm_id, parcelnumber])
                                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repaired", "Buffer(0) succeeded", current_time])
                                                except Exception as repair_insert_err:
                                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Insert Failed", str(repair_insert_err), current_time])
                                            else:
                                                logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Failed", "Still invalid after buffer(0)", current_time])
                                        except Exception as repair_err:
                                            logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Error", str(repair_err), current_time])
                                else:
                                    valid_count += 1
                                    # Optionally log valid geometries (uncomment if needed)
                                    # logwriter.writerow([oid, treefarm_id, parcelnumber, "Valid", "", current_time])

                            except Exception as row_process_err:
                                unreadable_count += 1
                                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                                logwriter.writerow([processed_count, "?", "?", "Row Processing Error", str(row_process_err), current_time])
                                print(f"❌ Error processing row {processed_count}: {row_process_err}")
                                continue

                    except Exception as cursor_iteration_err:
                        print(f"❌ Error during cursor iteration: {cursor_iteration_err}")
                        print(f"📊 Processed {processed_count} records before error")

                        # Log the cursor error
                        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        logwriter.writerow([processed_count, "CURSOR_ERROR", "CURSOR_ERROR", "Cursor Iteration Failed", str(cursor_iteration_err), current_time])

                        # Don't return error - continue with summary of what was processed
                        print("🔄 Continuing with summary of processed records...")

                # Clean up cursor
                try:
                    del search_cursor
                except:
                    pass

    except Exception as e:
        print(f"❌ Critical error during processing: {e}")
        return {"error": f"Critical error during processing: {e}"}

    # Final summary
    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n✅ Processing completed in {total_time/60:.1f} minutes")

    # Get actual record counts from output feature classes
    try:
        invalid_fc_count = int(arcpy.GetCount_management(invalid_fc).getOutput(0))
        repaired_fc_count = int(arcpy.GetCount_management(repaired_fc).getOutput(0)) if run_repair else 0
    except:
        invalid_fc_count = "Unknown"
        repaired_fc_count = "Unknown"

    summary = {
        "📊 Total Records Processed": processed_count,
        "✅ Valid Geometries": valid_count,
        "🔎 Invalid Geometries": invalid_count,
        "🧽 Repaired Geometries": repaired_count if run_repair else "Not run",
        "⚠️ Null Geometries": null_geometry_count,
        "🚫 Unreadable Features": unreadable_count,
        "⏱️ Processing Time (minutes)": round(total_time/60, 1),
        "📍 Invalid Features Output": invalid_fc,
        "📊 Invalid FC Record Count": invalid_fc_count,
        "🛠️ Repaired Features Output": repaired_fc if run_repair else "Not run",
        "📊 Repaired FC Record Count": repaired_fc_count if run_repair else "Not run",
        "📝 Diagnostics CSV": log_csv
    }

    print("\n📋 Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    # Performance statistics
    if total_time > 0:
        records_per_second = processed_count / total_time
        print(f"\n📈 Performance: {records_per_second:.1f} records/second")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        result = diagnose_and_repair_geometries(
            input_fc=r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
            output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
            run_repair=True
        )

        print(f"\n🎉 Script completed successfully!")

    except Exception as main_err:
        print(f"❌ Script failed with error: {main_err}")
        import traceback
        traceback.print_exc()

🚀 Starting geometry validation at 2025-07-10 07:55:31
📊 Total records to process: 12,189
🗺️ Spatial Reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
🏗️ Creating output feature classes...
📝 Creating diagnostics CSV and starting processing...

✅ Processing completed in 0.1 minutes

📋 Geometry Validation Summary:
  📊 Total Records Processed: 0
  ✅ Valid Geometries: 0
  🔎 Invalid Geometries: 0
  🧽 Repaired Geometries: 0
  ⚠️ Null Geometries: 0
  🚫 Unreadable Features: 0
  ⏱️ Processing Time (minutes): 0.1
  📍 Invalid Features Output: C:\temp\TreeFarm_Validation.gdb\InvalidGeometries
  📊 Invalid FC Record Count: 0
  🛠️ Repaired Features Output: C:\temp\TreeFarm_Validation.gdb\RepairedGeometries
  📊 Repaired FC Record Count: 0
  📝 Diagnostics CSV: C:\temp\GeometryDiagnostics.csv

📈 Performance: 0.0 records/second

🎉 Script completed successfully!


In [3]:
import arcpy
import os
import csv
import time
from datetime import datetime

def diagnose_and_repair_geometries_batch(input_fc, output_gdb, run_repair=True, batch_size=100):
    """
    Alternative approach using batch processing with WHERE clauses to handle problematic geometries.
    This method processes records in small batches and skips problematic ranges if needed.
    """
    print(f"🚀 Starting batch geometry validation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Get total record count and OID range
    try:
        total_records = int(arcpy.GetCount_management(input_fc).getOutput(0))
        print(f"📊 Total records to process: {total_records:,}")

        # Get OID field name and range
        desc = arcpy.Describe(input_fc)
        oid_field = desc.OIDFieldName
        print(f"🔑 OID Field: {oid_field}")

        # Get min/max OID values
        with arcpy.da.SearchCursor(input_fc, [oid_field]) as cursor:
            oids = [row[0] for row in cursor]
        min_oid = min(oids)
        max_oid = max(oids)
        print(f"📈 OID Range: {min_oid} to {max_oid}")

    except Exception as e:
        print(f"⚠️ Could not get record information: {e}")
        return {"error": f"Could not get record information: {e}"}

    # Create output geodatabase
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    # Get spatial reference
    try:
        spatial_ref = arcpy.Describe(input_fc).spatialReference
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")
    except Exception as e:
        print(f"❌ Failed to get spatial reference: {e}")
        return {"error": f"Failed to get spatial reference: {e}"}

    # Output paths
    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None
    log_csv = os.path.join(os.path.dirname(output_gdb), "GeometryDiagnostics_Batch.csv")

    # Create output feature classes
    print("🏗️ Creating output feature classes...")
    try:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

        arcpy.AddField_management(invalid_fc, "ORIG_OID", "LONG")
        arcpy.AddField_management(invalid_fc, "treefarm_id", "TEXT", field_length=50)
        arcpy.AddField_management(invalid_fc, "parcelnumber", "TEXT", field_length=50)
        arcpy.AddField_management(invalid_fc, "ERROR_TYPE", "TEXT", field_length=100)

        if run_repair:
            arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                                geometry_type="POLYGON", spatial_reference=spatial_ref)
            arcpy.AddField_management(repaired_fc, "ORIG_OID", "LONG")
            arcpy.AddField_management(repaired_fc, "treefarm_id", "TEXT", field_length=50)
            arcpy.AddField_management(repaired_fc, "parcelnumber", "TEXT", field_length=50)
            arcpy.AddField_management(repaired_fc, "REPAIR_METHOD", "TEXT", field_length=50)

    except Exception as e:
        print(f"❌ Failed to create output feature classes: {e}")
        return {"error": f"Failed to create output feature classes: {e}"}

    # Initialize counters
    invalid_count = 0
    repaired_count = 0
    unreadable_count = 0
    null_geometry_count = 0
    valid_count = 0
    processed_count = 0
    skipped_batches = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]

    print(f"📝 Starting batch processing (batch size: {batch_size})...")

    # Open CSV for logging
    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["BatchStart", "BatchEnd", "OID", "TreeFarmID", "ParcelNumber", "Status", "Details", "Timestamp"])

        # Create insert cursors
        invalid_insert_fields = ["SHAPE@", "ORIG_OID", "treefarm_id", "parcelnumber", "ERROR_TYPE"]
        repaired_insert_fields = ["SHAPE@", "ORIG_OID", "treefarm_id", "parcelnumber", "REPAIR_METHOD"]

        with arcpy.da.InsertCursor(invalid_fc, invalid_insert_fields) as invalid_writer, \
             (arcpy.da.InsertCursor(repaired_fc, repaired_insert_fields) if run_repair else None) as repaired_writer:

            # Process in batches
            current_oid = min_oid

            while current_oid <= max_oid:
                batch_end = min(current_oid + batch_size - 1, max_oid)
                batch_start_time = time.time()

                # Create WHERE clause for this batch
                where_clause = f"{oid_field} >= {current_oid} AND {oid_field} <= {batch_end}"

                print(f"🔄 Processing batch: OID {current_oid} to {batch_end}")

                try:
                    # Process this batch
                    with arcpy.da.SearchCursor(input_fc, fields, where_clause=where_clause) as batch_cursor:
                        batch_processed = 0
                        batch_invalid = 0
                        batch_valid = 0

                        for row in batch_cursor:
                            try:
                                processed_count += 1
                                batch_processed += 1

                                oid, shape, treefarm_id, parcelnumber = row
                                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                                # Handle null geometries
                                if shape is None:
                                    null_geometry_count += 1
                                    logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                      "Null Geometry", "Shape is None", current_time])
                                    continue

                                # Check geometry validity with error handling
                                try:
                                    is_valid = shape.isValid
                                except Exception as validity_err:
                                    unreadable_count += 1
                                    error_msg = f"Validity check failed: {validity_err}"
                                    logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                      "Validity Check Failed", error_msg, current_time])

                                    # Try to insert as invalid with error info
                                    try:
                                        invalid_writer.insertRow([None, oid, treefarm_id, parcelnumber, error_msg])
                                        invalid_count += 1
                                    except:
                                        pass
                                    continue

                                if not is_valid:
                                    invalid_count += 1
                                    batch_invalid += 1

                                    # Try to insert invalid geometry
                                    try:
                                        invalid_writer.insertRow([shape, oid, treefarm_id, parcelnumber, "Invalid Geometry"])
                                        logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                          "Invalid", "Failed isValid()", current_time])
                                    except Exception as insert_err:
                                        error_msg = f"Insert failed: {insert_err}"
                                        logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                          "Insert Failed", error_msg, current_time])

                                    # Attempt repair
                                    if run_repair:
                                        try:
                                            repaired_shape = shape.buffer(0)
                                            if repaired_shape and repaired_shape.isValid:
                                                repaired_count += 1
                                                try:
                                                    repaired_writer.insertRow([repaired_shape, oid, treefarm_id, parcelnumber, "Buffer(0)"])
                                                    logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                                      "Repaired", "Buffer(0) succeeded", current_time])
                                                except Exception as repair_insert_err:
                                                    logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                                      "Repair Insert Failed", str(repair_insert_err), current_time])
                                            else:
                                                logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                                  "Repair Failed", "Still invalid after buffer(0)", current_time])
                                        except Exception as repair_err:
                                            logwriter.writerow([current_oid, batch_end, oid, treefarm_id, parcelnumber,
                                                              "Repair Error", str(repair_err), current_time])
                                else:
                                    valid_count += 1
                                    batch_valid += 1

                            except Exception as row_err:
                                unreadable_count += 1
                                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                                logwriter.writerow([current_oid, batch_end, "?", "?", "?",
                                                  "Row Error", str(row_err), current_time])
                                continue

                        # Report batch completion
                        batch_time = time.time() - batch_start_time
                        print(f"  ✅ Batch completed: {batch_processed} records, {batch_invalid} invalid, {batch_valid} valid ({batch_time:.1f}s)")

                except Exception as batch_err:
                    skipped_batches += 1
                    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    print(f"  ❌ Batch failed: {batch_err}")
                    logwriter.writerow([current_oid, batch_end, "BATCH_ERROR", "BATCH_ERROR", "BATCH_ERROR",
                                      "Batch Failed", str(batch_err), current_time])

                # Move to next batch
                current_oid = batch_end + 1

                # Progress update
                elapsed = time.time() - start_time
                progress = (current_oid - min_oid) / (max_oid - min_oid + 1) * 100
                print(f"⏳ Overall Progress: {progress:.1f}% - Processed: {processed_count:,} - "
                      f"Invalid: {invalid_count:,} - Valid: {valid_count:,} - Elapsed: {elapsed/60:.1f}min")

    # Final summary
    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n✅ Batch processing completed in {total_time/60:.1f} minutes")

    # Get output record counts
    try:
        invalid_fc_count = int(arcpy.GetCount_management(invalid_fc).getOutput(0))
        repaired_fc_count = int(arcpy.GetCount_management(repaired_fc).getOutput(0)) if run_repair else 0
    except:
        invalid_fc_count = "Unknown"
        repaired_fc_count = "Unknown"

    summary = {
        "📊 Total Records Processed": processed_count,
        "✅ Valid Geometries": valid_count,
        "🔎 Invalid Geometries": invalid_count,
        "🧽 Repaired Geometries": repaired_count if run_repair else "Not run",
        "⚠️ Null Geometries": null_geometry_count,
        "🚫 Unreadable Features": unreadable_count,
        "⚠️ Skipped Batches": skipped_batches,
        "⏱️ Processing Time (minutes)": round(total_time/60, 1),
        "📍 Invalid Features Output": invalid_fc,
        "📊 Invalid FC Record Count": invalid_fc_count,
        "🛠️ Repaired Features Output": repaired_fc if run_repair else "Not run",
        "📊 Repaired FC Record Count": repaired_fc_count if run_repair else "Not run",
        "📝 Diagnostics CSV": log_csv
    }

    print("\n📋 Batch Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        # Try the batch approach with smaller batch size
        result = diagnose_and_repair_geometries_batch(
            input_fc=r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
            output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
            run_repair=True,
            batch_size=50  # Small batch size to isolate problematic records
        )

        print(f"\n🎉 Batch script completed successfully!")

    except Exception as main_err:
        print(f"❌ Script failed with error: {main_err}")
        import traceback
        traceback.print_exc()

🚀 Starting batch geometry validation at 2025-07-10 07:56:11
📊 Total records to process: 12,189
🔑 OID Field: OBJECTID
📈 OID Range: 1 to 682694
🗺️ Spatial Reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
🏗️ Creating output feature classes...
📝 Starting batch processing (batch size: 50)...
🔄 Processing batch: OID 1 to 50
  ✅ Batch completed: 49 records, 0 invalid, 0 valid (0.0s)
⏳ Overall Progress: 0.0% - Processed: 49 - Invalid: 49 - Valid: 0 - Elapsed: 0.2min
🔄 Processing batch: OID 51 to 100
  ✅ Batch completed: 47 records, 0 invalid, 0 valid (0.0s)
⏳ Overall Progress: 0.0% - Processed: 96 - Invalid: 96 - Valid: 0 - Elapsed: 0.2min
🔄 Processing batch: OID 101 to 150
  ✅ Batch completed: 46 records, 0 invalid, 0 valid (0.0s)
⏳ Overall Progress: 0.0% - Processed: 142 - Invalid: 142 - Valid: 0 - Elapsed: 0.2min
🔄 Processing batch: OID 151 to 200
  ✅ Batch completed: 45 records, 0 invalid, 0 valid (0.0s)
⏳ Overall Progress: 0.0% - Processed: 187 - Invalid: 187 - Valid: 0 - Elapsed: 0.2min

In [9]:
import arcpy
import os
import csv
import time
from datetime import datetime

def create_clean_dataset_fixed_fields(input_fc, output_gdb):
    """
    Create clean dataset with proper field handling - exclude computed geometry fields.
    """
    print(f"🚀 Starting corrected field copy at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Create output geodatabase
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    # Get spatial reference and field info
    try:
        spatial_ref = arcpy.Describe(input_fc).spatialReference
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")

        # Get field information - exclude computed fields
        fields = arcpy.ListFields(input_fc)

        # Filter out system and computed fields
        data_fields = []
        for field in fields:
            # Skip system fields and computed geometry fields
            if (field.type not in ['OID', 'Geometry'] and
                not field.name.startswith('Shape') and
                not field.name.startswith('SHAPE.') and
                field.name not in ['OBJECTID', 'GlobalID']):
                data_fields.append(field.name)

        print(f"📋 Data fields to copy: {', '.join(data_fields)}")

        if not data_fields:
            print("⚠️ No copyable data fields found, will copy geometry only")

    except Exception as e:
        print(f"❌ Failed to get dataset information: {e}")
        return {"error": f"Failed to get dataset information: {e}"}

    # Output paths
    clean_fc = os.path.join(output_gdb, "CleanTreeFarm")
    log_csv = os.path.join(os.path.dirname(output_gdb), "CorrectedCopy_Log.csv")

    # Create clean feature class
    print("🏗️ Creating clean feature class structure...")
    try:
        arcpy.CreateFeatureclass_management(
            output_gdb, "CleanTreeFarm", "POLYGON",
            spatial_reference=spatial_ref
        )

        # Add only the valid data fields
        for field_name in data_fields:
            try:
                field_obj = next(f for f in fields if f.name == field_name)
                arcpy.AddField_management(
                    clean_fc, field_name, field_obj.type,
                    field_precision=field_obj.precision if hasattr(field_obj, 'precision') else None,
                    field_scale=field_obj.scale if hasattr(field_obj, 'scale') else None,
                    field_length=field_obj.length if hasattr(field_obj, 'length') else None
                )
                print(f"  ✅ Added field: {field_name} ({field_obj.type})")
            except Exception as field_err:
                print(f"  ⚠️ Could not add field {field_name}: {field_err}")
                # Remove from data_fields if it can't be added
                if field_name in data_fields:
                    data_fields.remove(field_name)

    except Exception as e:
        print(f"❌ Failed to create clean feature class: {e}")
        return {"error": f"Failed to create clean feature class: {e}"}

    # Prepare fields for cursors - only include valid fields
    search_fields = ["OID@", "SHAPE@"] + data_fields
    insert_fields = ["SHAPE@"] + data_fields

    print(f"🔍 Using search fields: {search_fields}")
    print(f"📝 Using insert fields: {insert_fields}")

    # Initialize counters
    copied_count = 0
    skipped_count = 0
    error_count = 0
    processed_count = 0

    # Open log file
    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["OID", "Status", "Details", "Timestamp"])

        print(f"📝 Starting record-by-record copy...")

        # Create insert cursor for clean dataset
        with arcpy.da.InsertCursor(clean_fc, insert_fields) as insert_cursor:

            # Process records individually to handle problematic ones
            try:
                # First, try to get total count for progress tracking
                try:
                    total_count = int(arcpy.GetCount_management(input_fc).getOutput(0))
                    print(f"📊 Target records to copy: {total_count:,}")
                except:
                    total_count = None

                # Use a cursor to process records one by one
                with arcpy.da.SearchCursor(input_fc, search_fields) as search_cursor:

                    for row in search_cursor:
                        try:
                            processed_count += 1
                            oid = row[0]
                            shape = row[1]
                            data_values = row[2:] if len(row) > 2 else []

                            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                            # Skip if shape is None
                            if shape is None:
                                skipped_count += 1
                                logwriter.writerow([oid, "Skipped", "Null geometry", current_time])
                                continue

                            # Check if geometry has basic validity for copying
                            try:
                                # Try to access basic geometry properties
                                _ = shape.type  # This will fail if geometry is completely corrupted

                                # Try to insert the record
                                if data_fields:
                                    insert_row = [shape] + list(data_values)
                                else:
                                    insert_row = [shape]

                                insert_cursor.insertRow(insert_row)
                                copied_count += 1

                                # Log occasional successes
                                if copied_count <= 5 or copied_count % 1000 == 0:
                                    logwriter.writerow([oid, "Copied", "Successfully copied", current_time])

                            except Exception as geom_err:
                                error_count += 1
                                error_msg = str(geom_err)

                                if "number of points is less than required" in error_msg:
                                    logwriter.writerow([oid, "Skipped", "Invalid geometry - insufficient points", current_time])
                                    skipped_count += 1
                                else:
                                    logwriter.writerow([oid, "Error", error_msg[:100], current_time])

                                # Print first few errors
                                if error_count <= 10:
                                    print(f"  ❌ Error with OID {oid}: {error_msg[:100]}")
                                elif error_count == 11:
                                    print(f"  ⚠️ Additional errors suppressed...")

                                continue

                            # Progress reporting
                            if processed_count % 1000 == 0:
                                elapsed = time.time() - start_time
                                if total_count:
                                    progress = (processed_count / total_count) * 100
                                    print(f"⏳ Progress: {processed_count:,}/{total_count:,} ({progress:.1f}%) - "
                                          f"Copied: {copied_count:,} - Skipped: {skipped_count:,} - "
                                          f"Errors: {error_count:,} - Elapsed: {elapsed/60:.1f}min")
                                else:
                                    print(f"⏳ Processed: {processed_count:,} - Copied: {copied_count:,} - "
                                          f"Skipped: {skipped_count:,} - Errors: {error_count:,}")

                        except Exception as row_err:
                            error_count += 1
                            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                            # This catches cursor-level errors
                            error_msg = str(row_err)
                            logwriter.writerow([processed_count, "Cursor Error", error_msg[:100], current_time])

                            if "number of points is less than required" in error_msg:
                                print(f"  ❌ Cursor error at record {processed_count}: Invalid geometry")
                                # Try to continue
                                continue
                            else:
                                print(f"  ❌ Unexpected cursor error: {error_msg}")
                                break  # Stop if we get unexpected errors

            except Exception as cursor_err:
                print(f"❌ Critical cursor error: {cursor_err}")

                # If the main cursor fails, try a different approach
                print("🔄 Trying alternative single-record approach...")

                # Get list of OIDs and process them individually
                try:
                    oid_field = arcpy.Describe(input_fc).OIDFieldName
                    with arcpy.da.SearchCursor(input_fc, [oid_field]) as oid_cursor:
                        oid_list = [row[0] for row in oid_cursor]

                    print(f"📊 Processing {len(oid_list):,} records individually...")

                    for oid in oid_list:
                        try:
                            where_clause = f"{oid_field} = {oid}"
                            with arcpy.da.SearchCursor(input_fc, search_fields, where_clause) as single_cursor:
                                for row in single_cursor:
                                    processed_count += 1
                                    shape = row[1]
                                    data_values = row[2:] if len(row) > 2 else []

                                    if shape is not None:
                                        try:
                                            if data_fields:
                                                insert_row = [shape] + list(data_values)
                                            else:
                                                insert_row = [shape]
                                            insert_cursor.insertRow(insert_row)
                                            copied_count += 1
                                        except:
                                            skipped_count += 1
                                    else:
                                        skipped_count += 1

                                    if processed_count % 1000 == 0:
                                        print(f"  ⏳ Individual processing: {processed_count:,} - Copied: {copied_count:,}")

                        except Exception as single_err:
                            # Skip problematic individual records
                            if "number of points is less than required" not in str(single_err):
                                print(f"  ⚠️ Skipping OID {oid}: {single_err}")
                            continue

                except Exception as alt_err:
                    print(f"❌ Alternative approach also failed: {alt_err}")

    # Verify the clean dataset
    try:
        final_count = int(arcpy.GetCount_management(clean_fc).getOutput(0))
        print(f"✅ Clean dataset verification: {final_count:,} records")
    except Exception as e:
        print(f"⚠️ Could not verify clean dataset: {e}")
        final_count = "Unknown"

    # Final summary
    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n✅ Corrected field copy completed in {total_time/60:.1f} minutes")

    summary = {
        "📊 Total Records Processed": processed_count,
        "✅ Records Successfully Copied": copied_count,
        "⚠️ Records Skipped": skipped_count,
        "❌ Records with Errors": error_count,
        "📊 Final Clean Dataset Count": final_count,
        "📈 Success Rate": f"{(copied_count/processed_count*100):.1f}%" if processed_count > 0 else "0%",
        "⏱️ Processing Time (minutes)": round(total_time/60, 1),
        "🧹 Clean Dataset": clean_fc,
        "📝 Detailed Log": log_csv
    }

    print("\n📋 Corrected Copy Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    if copied_count > 0:
        print(f"\n💡 Success! Next Steps:")
        print(f"  1. ✅ Use clean dataset: {clean_fc}")
        print(f"  2. 🔧 Run geometry repair tools on clean dataset")
        print(f"  3. 📊 Clean dataset should work with all ArcPy tools")

        # Test the clean dataset
        try:
            print(f"\n🧪 Testing clean dataset...")
            test_cursor = arcpy.da.SearchCursor(clean_fc, ["OID@", "SHAPE@"])
            test_row = next(test_cursor)
            del test_cursor
            print(f"  ✅ Clean dataset cursor test: SUCCESS")
        except Exception as test_err:
            print(f"  ❌ Clean dataset cursor test failed: {test_err}")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        result = create_clean_dataset_fixed_fields(
            input_fc=r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
            output_gdb=r"C:\temp\TreeFarm_Clean_Fixed.gdb"
        )

        print(f"\n🎉 Fixed field copy completed!")

        if result.get("✅ Records Successfully Copied", 0) > 0:
            print(f"\n🎯 Ready for geometry repair!")
            print(f"  📂 Clean dataset: {result['🧹 Clean Dataset']}")
            print(f"  📊 Records available: {result['✅ Records Successfully Copied']:,}")

    except Exception as main_err:
        print(f"❌ Script failed with error: {main_err}")
        import traceback
        traceback.print_exc()

🚀 Starting corrected field copy at 2025-07-10 08:16:23
📁 Creating output geodatabase: C:\temp\TreeFarm_Clean_Fixed.gdb
🗺️ Spatial Reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
📋 Data fields to copy: treefarm_id, parcelnumber
🏗️ Creating clean feature class structure...
  ✅ Added field: treefarm_id (Integer)
  ✅ Added field: parcelnumber (SmallInteger)
🔍 Using search fields: ['OID@', 'SHAPE@', 'treefarm_id', 'parcelnumber']
📝 Using insert fields: ['SHAPE@', 'treefarm_id', 'parcelnumber']
📝 Starting record-by-record copy...
📊 Target records to copy: 12,189
❌ Critical cursor error: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
🔄 Trying alternative single-record approach...
📊 Processing 12,189 records individually...
  ⏳ Individual processing: 1,000 - Copied: 1,000
  ⏳ Individual processing: 2,000 - Copied: 2,000
  ⏳ Individual processing: 3,000 - Copied: 2,999
  ⏳ Individual processing: 4,000 - Copied: 3,999
  ⏳ Individual processing: 5,0

In [ ]:
import arcpy
import os
import csv
import time
from datetime import datetime

def repair_clean_dataset(clean_fc, output_gdb):
    """
    Run comprehensive geometry repair on the clean dataset.
    """
    print(f"🚀 Starting geometry repair on clean dataset at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Get record count
    try:
        total_records = int(arcpy.GetCount_management(clean_fc).getOutput(0))
        print(f"📊 Clean dataset records: {total_records:,}")
    except Exception as e:
        print(f"⚠️ Could not get record count: {e}")
        total_records = None

    # Create output geodatabase
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    # Get spatial reference
    try:
        spatial_ref = arcpy.Describe(clean_fc).spatialReference
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")
    except Exception as e:
        print(f"❌ Failed to get spatial reference: {e}")
        return {"error": f"Failed to get spatial reference: {e}"}

    # Output paths
    validation_before = os.path.join(output_gdb, "ValidationBefore")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries")
    validation_after = os.path.join(output_gdb, "ValidationAfter")
    log_csv = os.path.join(os.path.dirname(output_gdb), "GeometryRepair_Final.csv")

    print("\n" + "="*60)
    print("🔍 STEP 1: VALIDATE GEOMETRIES BEFORE REPAIR")
    print("="*60)

    try:
        # Check Geometry to get detailed validation report
        print("📋 Running Check Geometry tool...")
        arcpy.management.CheckGeometry(clean_fc, validation_before)

        validation_count = int(arcpy.GetCount_management(validation_before).getOutput(0))
        print(f"📊 Geometry issues found: {validation_count:,}")

        if validation_count > 0:
            print("📝 Sample geometry errors:")
            with arcpy.da.SearchCursor(validation_before, ["FEATURE_ID", "PROBLEM"]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 10:  # Show first 10 errors
                        print(f"  • Feature {row[0]}: {row[1]}")
                    else:
                        break

            # Count error types
            error_types = {}
            with arcpy.da.SearchCursor(validation_before, ["PROBLEM"]) as cursor:
                for row in cursor:
                    problem = row[0]
                    error_types[problem] = error_types.get(problem, 0) + 1

            print(f"\n📈 Error type breakdown:")
            for error, count in sorted(error_types.items(), key=lambda x: x[1], reverse=True):
                print(f"  • {error}: {count:,} occurrences")
        else:
            print("✅ No geometry errors found!")

    except Exception as e:
        print(f"❌ Check Geometry failed: {e}")
        validation_count = 0

    print("\n" + "="*60)
    print("🛠️ STEP 2: REPAIR GEOMETRIES")
    print("="*60)

    if validation_count > 0:
        try:
            # Create a copy for repair
            print("📋 Creating working copy for repair...")
            arcpy.CopyFeatures_management(clean_fc, repaired_fc)

            # Run Repair Geometry tool
            print("🔧 Running Repair Geometry tool...")
            repair_start = time.time()

            repair_result = arcpy.management.RepairGeometry(repaired_fc, "DELETE_NULL")

            repair_time = time.time() - repair_start
            print(f"⏱️ Repair completed in {repair_time:.1f} seconds")

            # Get the repair results
            repair_messages = repair_result.getMessages()
            print(f"📋 Repair Geometry results:")
            for line in repair_messages.split('\n'):
                if line.strip():
                    print(f"  {line}")

        except Exception as e:
            print(f"❌ Repair Geometry tool failed: {e}")
            # Create copy without repair
            arcpy.CopyFeatures_management(clean_fc, repaired_fc)
            repair_messages = f"Repair failed: {e}"
    else:
        print("✅ No repairs needed - copying clean dataset")
        arcpy.CopyFeatures_management(clean_fc, repaired_fc)
        repair_messages = "No repairs needed"

    print("\n" + "="*60)
    print("✅ STEP 3: VALIDATE REPAIRED GEOMETRIES")
    print("="*60)

    try:
        # Re-validate the repaired geometries
        print("📋 Running post-repair validation...")
        arcpy.management.CheckGeometry(repaired_fc, validation_after)

        validation_after_count = int(arcpy.GetCount_management(validation_after).getOutput(0))
        print(f"📊 Remaining issues after repair: {validation_after_count:,}")

        if validation_after_count > 0:
            print("📝 Remaining geometry errors:")
            with arcpy.da.SearchCursor(validation_after, ["FEATURE_ID", "PROBLEM"]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 10:
                        print(f"  • Feature {row[0]}: {row[1]}")
                    else:
                        break

            # Count remaining error types
            remaining_error_types = {}
            with arcpy.da.SearchCursor(validation_after, ["PROBLEM"]) as cursor:
                for row in cursor:
                    problem = row[0]
                    remaining_error_types[problem] = remaining_error_types.get(problem, 0) + 1

            print(f"\n📈 Remaining error type breakdown:")
            for error, count in sorted(remaining_error_types.items(), key=lambda x: x[1], reverse=True):
                print(f"  • {error}: {count:,} occurrences")
        else:
            print("🎉 All geometry errors successfully repaired!")

    except Exception as e:
        print(f"❌ Post-repair validation failed: {e}")
        validation_after_count = "Unknown"

    print("\n" + "="*60)
    print("📊 STEP 4: MANUAL VALIDATION & STATISTICS")
    print("="*60)

    # Manual validation of repaired geometries
    print("📋 Performing manual geometry validation...")

    valid_count = 0
    invalid_count = 0
    null_count = 0
    manual_processed = 0

    try:
        with arcpy.da.SearchCursor(repaired_fc, ["OID@", "SHAPE@"]) as cursor:
            for row in cursor:
                manual_processed += 1
                oid, shape = row

                if shape is None:
                    null_count += 1
                else:
                    try:
                        if shape.isValid:
                            valid_count += 1
                        else:
                            invalid_count += 1
                    except:
                        invalid_count += 1

                if manual_processed % 2000 == 0:
                    print(f"  ⏳ Validated {manual_processed:,} records...")

    except Exception as e:
        print(f"⚠️ Manual validation failed: {e}")

    print(f"📊 Manual validation results:")
    print(f"  ✅ Valid geometries: {valid_count:,}")
    print(f"  ❌ Invalid geometries: {invalid_count:,}")
    print(f"  ⚠️ Null geometries: {null_count:,}")

    print("\n" + "="*60)
    print("📝 STEP 5: CREATE DETAILED REPORT")
    print("="*60)

    # Create detailed CSV report
    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["Phase", "Metric", "Value", "Notes", "Timestamp"])

        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Log all statistics
        logwriter.writerow(["Input", "Total Records", total_records, "Clean dataset", current_time])
        logwriter.writerow(["Before Repair", "Geometry Issues", validation_count, "From Check Geometry", current_time])
        logwriter.writerow(["After Repair", "Remaining Issues", validation_after_count, "From Check Geometry", current_time])
        logwriter.writerow(["Manual Validation", "Valid Geometries", valid_count, "Manual isValid check", current_time])
        logwriter.writerow(["Manual Validation", "Invalid Geometries", invalid_count, "Manual isValid check", current_time])
        logwriter.writerow(["Manual Validation", "Null Geometries", null_count, "Manual null check", current_time])

        # Log repair messages
        for line in repair_messages.split('\n'):
            if line.strip():
                logwriter.writerow(["Repair", "Message", line.strip(), "From Repair Geometry tool", current_time])

    # Calculate repair success metrics
    if validation_count > 0:
        if isinstance(validation_after_count, int):
            issues_repaired = validation_count - validation_after_count
            repair_success_rate = (issues_repaired / validation_count) * 100
        else:
            issues_repaired = "Unknown"
            repair_success_rate = "Unknown"
    else:
        issues_repaired = 0
        repair_success_rate = 100.0

    # Get final record count
    try:
        final_count = int(arcpy.GetCount_management(repaired_fc).getOutput(0))
    except:
        final_count = "Unknown"

    # Final summary
    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n✅ Comprehensive geometry repair completed in {total_time/60:.1f} minutes")

    summary = {
        "📊 Input Records": total_records,
        "📊 Final Output Records": final_count,
        "🔍 Geometry Issues Before Repair": validation_count,
        "🔍 Geometry Issues After Repair": validation_after_count,
        "🛠️ Issues Successfully Repaired": issues_repaired,
        "📈 Repair Success Rate": f"{repair_success_rate:.1f}%" if isinstance(repair_success_rate, (int, float)) else repair_success_rate,
        "✅ Valid Geometries (Manual)": valid_count,
        "❌ Invalid Geometries (Manual)": invalid_count,
        "⚠️ Null Geometries": null_count,
        "⏱️ Total Processing Time (minutes)": round(total_time/60, 1),
        "🛠️ Repaired Dataset": repaired_fc,
        "🔍 Validation Before": validation_before,
        "🔍 Validation After": validation_after,
        "📝 Detailed Report": log_csv
    }

    print("\n" + "="*60)
    print("📋 FINAL GEOMETRY REPAIR SUMMARY")
    print("="*60)

    for k, v in summary.items():
        print(f"  {k}: {v}")

    # Provide recommendations
    print(f"\n💡 RECOMMENDATIONS:")

    if isinstance(repair_success_rate, (int, float)):
        if repair_success_rate >= 90:
            print(f"  🎉 EXCELLENT! {repair_success_rate:.1f}% repair success rate")
            print(f"  ✅ Your dataset is now ready for production use")
        elif repair_success_rate >= 70:
            print(f"  👍 GOOD! {repair_success_rate:.1f}% repair success rate")
            print(f"  ✅ Most issues resolved - review remaining invalid geometries")
        elif repair_success_rate >= 40:
            print(f"  ⚠️ MODERATE: {repair_success_rate:.1f}% repair success rate")
            print(f"  🔧 Consider manual editing for remaining problematic geometries")
        else:
            print(f"  ❌ LOW: {repair_success_rate:.1f}% repair success rate")
            print(f"  🛠️ May need more aggressive repair strategies or manual intervention")

    if invalid_count > 0:
        print(f"  📝 Review validation tables for specific error patterns")
        print(f"  🔧 Consider excluding invalid geometries from analysis if minimal impact")

    print(f"\n🎯 NEXT STEPS:")
    print(f"  1. Use repaired dataset: {repaired_fc}")
    print(f"  2. Review validation reports for quality assurance")
    print(f"  3. Test dataset with your analysis workflows")
    print(f"  4. Consider adding computed fields (area, length) if needed")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        result = repair_clean_dataset(
            clean_fc=r"C:\temp\TreeFarm_Clean_Fixed.gdb\CleanTreeFarm",
            output_gdb=r"C:\temp\TreeFarm_Final_Repaired.gdb"
        )

        print(f"\n🎉 GEOMETRY REPAIR PROCESS COMPLETED SUCCESSFULLY!")

    except Exception as main_err:
        print(f"❌ Script failed with error: {main_err}")
        import traceback
        traceback.print_exc()

In [10]:
import arcpy
import os
import time
from datetime import datetime

def investigate_geometry_validation_discrepancy(input_fc, output_gdb):
    """
    Investigate why ArcPy is reporting so many invalid geometries when SQL Server only found 18.
    Compare different validation methods to identify the source of discrepancy.
    """
    print(f"🔍 Starting geometry validation investigation at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Create output geodatabase
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    # Get basic dataset info
    try:
        total_records = int(arcpy.GetCount_management(input_fc).getOutput(0))
        spatial_ref = arcpy.Describe(input_fc).spatialReference
        print(f"📊 Total records: {total_records:,}")
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")
        print(f"📍 Coordinate System: {spatial_ref.type}")
    except Exception as e:
        print(f"❌ Failed to get basic info: {e}")
        return {"error": f"Failed to get basic info: {e}"}

    print("\n" + "="*60)
    print("🧪 TEST 1: SAMPLE GEOMETRY VALIDATION")
    print("="*60)

    # Test a small sample of geometries with different validation methods
    sample_size = 50
    print(f"📋 Testing first {sample_size} geometries with multiple validation methods...")

    validation_results = {
        "arcpy_isValid": {"valid": 0, "invalid": 0, "error": 0},
        "shape_properties": {"accessible": 0, "not_accessible": 0},
        "basic_operations": {"successful": 0, "failed": 0}
    }

    try:
        with arcpy.da.SearchCursor(input_fc, ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]) as cursor:
            for i, row in enumerate(cursor):
                if i >= sample_size:
                    break

                oid, shape, treefarm_id, parcelnumber = row
                print(f"🔍 Testing OID {oid} (treefarm_id: {treefarm_id})")

                # Test 1: ArcPy isValid
                try:
                    if shape is None:
                        print(f"  ⚠️ Null geometry")
                        continue

                    is_valid = shape.isValid
                    if is_valid:
                        validation_results["arcpy_isValid"]["valid"] += 1
                        print(f"  ✅ ArcPy isValid: TRUE")
                    else:
                        validation_results["arcpy_isValid"]["invalid"] += 1
                        print(f"  ❌ ArcPy isValid: FALSE")

                        # Try to get more details about the invalid geometry
                        try:
                            area = shape.area
                            length = shape.length
                            part_count = shape.partCount
                            point_count = shape.pointCount
                            print(f"      📊 Area: {area:.2f}, Length: {length:.2f}")
                            print(f"      📊 Parts: {part_count}, Points: {point_count}")
                        except Exception as detail_err:
                            print(f"      ❌ Cannot access geometry details: {detail_err}")

                except Exception as valid_err:
                    validation_results["arcpy_isValid"]["error"] += 1
                    print(f"  ❌ ArcPy isValid failed: {valid_err}")

                # Test 2: Basic shape properties
                try:
                    _ = shape.area
                    _ = shape.length
                    _ = shape.type
                    validation_results["shape_properties"]["accessible"] += 1
                    print(f"  ✅ Shape properties accessible")
                except Exception as prop_err:
                    validation_results["shape_properties"]["not_accessible"] += 1
                    print(f"  ❌ Shape properties not accessible: {prop_err}")

                # Test 3: Basic geometric operations
                try:
                    _ = shape.centroid
                    _ = shape.extent
                    validation_results["basic_operations"]["successful"] += 1
                    print(f"  ✅ Basic operations successful")
                except Exception as op_err:
                    validation_results["basic_operations"]["failed"] += 1
                    print(f"  ❌ Basic operations failed: {op_err}")

                print()  # Blank line for readability

    except Exception as e:
        print(f"❌ Sample testing failed: {e}")

    print("\n" + "="*60)
    print("📊 SAMPLE VALIDATION RESULTS")
    print("="*60)

    for method, results in validation_results.items():
        print(f"\n🔍 {method}:")
        for status, count in results.items():
            print(f"  {status}: {count}")

    print("\n" + "="*60)
    print("🧪 TEST 2: ARCGIS CHECK GEOMETRY vs SQL SERVER")
    print("="*60)

    # Run ArcGIS Check Geometry tool for comparison
    validation_table = os.path.join(output_gdb, "ArcGIS_CheckGeometry")

    try:
        print("📋 Running ArcGIS Check Geometry tool...")
        arcpy.management.CheckGeometry(input_fc, validation_table)

        arcgis_issues = int(arcpy.GetCount_management(validation_table).getOutput(0))
        print(f"📊 ArcGIS Check Geometry found: {arcgis_issues:,} issues")

        if arcgis_issues > 0:
            print("📝 Sample ArcGIS geometry issues:")
            with arcpy.da.SearchCursor(validation_table, ["FEATURE_ID", "PROBLEM"]) as cursor:
                for i, row in enumerate(cursor):
                    if i < 10:
                        print(f"  • Feature {row[0]}: {row[1]}")
                    else:
                        break

    except Exception as e:
        print(f"❌ ArcGIS Check Geometry failed: {e}")
        arcgis_issues = "Error"

    print("\n" + "="*60)
    print("🧪 TEST 3: COORDINATE SYSTEM ANALYSIS")
    print("="*60)

    # Check if coordinate system issues are causing false positives
    print("🗺️ Analyzing coordinate system...")

    try:
        desc = arcpy.Describe(input_fc)
        sr = desc.spatialReference

        print(f"📍 Coordinate System Details:")
        print(f"  Name: {sr.name}")
        print(f"  Type: {sr.type}")
        print(f"  Units: {sr.linearUnitName}")
        print(f"  Authority: {sr.factoryCode}")

        # Check for known problematic coordinate systems
        if "Web_Mercator" in sr.name:
            print("  ⚠️ Web Mercator detected - may cause precision issues")

        if sr.type == "Geographic":
            print("  ⚠️ Geographic coordinate system - may cause validation issues")

    except Exception as e:
        print(f"❌ Coordinate system analysis failed: {e}")

    print("\n" + "="*60)
    print("🧪 TEST 4: COMPARE SPECIFIC RECORDS FROM SQL")
    print("="*60)

    # Test the specific OIDs that SQL Server identified as problematic
    sql_problematic_oids = [681161, 681171, 681201, 681233, 681240, 681622, 681684, 682079, 682090, 682105, 682161, 682169, 682251, 682277, 682297, 682301, 682441, 682488, 682511]

    print(f"🔍 Testing the {len(sql_problematic_oids)} OIDs that SQL Server identified as invalid...")

    sql_validation_results = {"found": 0, "not_found": 0, "valid_in_arcpy": 0, "invalid_in_arcpy": 0}

    try:
        oid_field = arcpy.Describe(input_fc).OIDFieldName

        for oid in sql_problematic_oids[:10]:  # Test first 10
            where_clause = f"{oid_field} = {oid}"

            try:
                with arcpy.da.SearchCursor(input_fc, ["OID@", "SHAPE@", "treefarm_id"], where_clause) as cursor:
                    row = next(cursor, None)
                    if row:
                        sql_validation_results["found"] += 1
                        oid_val, shape, treefarm_id = row

                        if shape is not None:
                            try:
                                is_valid = shape.isValid
                                if is_valid:
                                    sql_validation_results["valid_in_arcpy"] += 1
                                    print(f"  🔍 OID {oid} (treefarm_id {treefarm_id}): SQL=Invalid, ArcPy=Valid ⚠️")
                                else:
                                    sql_validation_results["invalid_in_arcpy"] += 1
                                    print(f"  ✅ OID {oid} (treefarm_id {treefarm_id}): SQL=Invalid, ArcPy=Invalid ✓")
                            except Exception as valid_err:
                                print(f"  ❌ OID {oid}: Cannot check validity - {valid_err}")
                    else:
                        sql_validation_results["not_found"] += 1
                        print(f"  ❌ OID {oid}: Not found in dataset")

            except Exception as oid_err:
                print(f"  ❌ Error checking OID {oid}: {oid_err}")

    except Exception as e:
        print(f"❌ SQL OID validation failed: {e}")

    print(f"\n📊 SQL OID Validation Results:")
    for status, count in sql_validation_results.items():
        print(f"  {status}: {count}")

    print("\n" + "="*60)
    print("🔍 INVESTIGATION SUMMARY")
    print("="*60)

    # Summary and hypothesis
    summary = {
        "Total Records": total_records,
        "ArcGIS Check Geometry Issues": arcgis_issues,
        "SQL Server Issues": 18,
        "Sample ArcPy Invalid": validation_results["arcpy_isValid"]["invalid"],
        "Sample ArcPy Valid": validation_results["arcpy_isValid"]["valid"],
        "Coordinate System": spatial_ref.name,
        "Investigation Time": round((time.time() - start_time)/60, 1)
    }

    print("📋 Key Findings:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    print(f"\n💡 HYPOTHESIS:")

    # Determine most likely cause
    if arcgis_issues < 100:
        print("  ✅ ArcGIS Check Geometry agrees with SQL Server")
        print("  🔍 The 11,899 'invalid' count was likely due to:")
        print("     • Coordinate system precision issues")
        print("     • ArcPy isValid() being overly strict")
        print("     • Processing errors in our validation script")
    elif arcgis_issues > 10000:
        print("  ❌ ArcGIS Check Geometry also finds many issues")
        print("  🔍 There may be widespread but minor geometry problems")
        print("     • SQL Server tolerance vs ArcGIS tolerance differences")
        print("     • Coordinate system validation differences")
    else:
        print("  ⚠️ Moderate number of geometry issues confirmed")
        print("  🔍 SQL Server found the most serious issues (18)")
        print("  🔍 ArcGIS found additional minor issues")

    print(f"\n🎯 RECOMMENDATION:")
    print("  1. Focus on the 18 records SQL Server identified")
    print("  2. Use ArcGIS Check Geometry results as authoritative")
    print("  3. Ignore minor coordinate precision 'errors'")
    print("  4. Test repairs on a small subset first")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        result = investigate_geometry_validation_discrepancy(
            input_fc=r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
            output_gdb=r"C:\temp\GeometryInvestigation.gdb"
        )

        print(f"\n🎉 Investigation completed!")

    except Exception as main_err:
        print(f"❌ Investigation failed: {main_err}")
        import traceback
        traceback.print_exc()

🔍 Starting geometry validation investigation at 2025-07-10 08:31:51
📁 Creating output geodatabase: C:\temp\GeometryInvestigation.gdb
📊 Total records: 12,189
🗺️ Spatial Reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
📍 Coordinate System: Projected

🧪 TEST 1: SAMPLE GEOMETRY VALIDATION
📋 Testing first 50 geometries with multiple validation methods...
❌ Sample testing failed: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]

📊 SAMPLE VALIDATION RESULTS

🔍 arcpy_isValid:
  valid: 0
  invalid: 0
  error: 0

🔍 shape_properties:
  accessible: 0
  not_accessible: 0

🔍 basic_operations:
  successful: 0
  failed: 0

🧪 TEST 2: ARCGIS CHECK GEOMETRY vs SQL SERVER
📋 Running ArcGIS Check Geometry tool...
📊 ArcGIS Check Geometry found: 20 issues
📝 Sample ArcGIS geometry issues:
  • Feature 681161: Shape has a structural error! (-151) A polygon outer shell does not completely enclose all donuts for the part
  • Feature 681171: Failed to select shape! (-148

In [12]:
import arcpy
import os
import csv
import time
from datetime import datetime

def targeted_geometry_fix(input_fc, output_gdb):
    """
    Targeted fix for the ~20 actual geometry problems identified by ArcGIS Check Geometry.
    This is a much more reasonable and accurate approach.
    """
    print(f"🚀 Starting targeted geometry fix at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("🎯 Focusing on the ~20 actual geometry problems (not 11,899!)")
    start_time = time.time()

    arcpy.env.overwriteOutput = True

    # Get basic info
    try:
        total_records = int(arcpy.GetCount_management(input_fc).getOutput(0))
        spatial_ref = arcpy.Describe(input_fc).spatialReference
        print(f"📊 Total records: {total_records:,}")
        print(f"🗺️ Spatial Reference: {spatial_ref.name}")
    except Exception as e:
        print(f"❌ Failed to get basic info: {e}")
        return {"error": f"Failed to get basic info: {e}"}

    # Create output geodatabase
    if not arcpy.Exists(output_gdb):
        print(f"📁 Creating output geodatabase: {output_gdb}")
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    # Output paths
    validation_table = os.path.join(output_gdb, "GeometryIssues")
    clean_fc = os.path.join(output_gdb, "CleanTreeFarm")
    repaired_fc = os.path.join(output_gdb, "RepairedTreeFarm")
    broken_geometries_fc = os.path.join(output_gdb, "BrokenGeometries")
    log_csv = os.path.join(os.path.dirname(output_gdb), "TargetedRepair_Log.csv")

    print("\n" + "="*60)
    print("🔍 STEP 1: IDENTIFY ACTUAL GEOMETRY PROBLEMS")
    print("="*60)

    # Run Check Geometry to identify the actual problems
    try:
        print("📋 Running ArcGIS Check Geometry (authoritative validation)...")
        arcpy.management.CheckGeometry(input_fc, validation_table)

        actual_issues = int(arcpy.GetCount_management(validation_table).getOutput(0))
        print(f"📊 Actual geometry issues found: {actual_issues}")

        if actual_issues == 0:
            print("🎉 No geometry issues found! Your dataset is clean.")
            # Just copy the input to output
            arcpy.CopyFeatures_management(input_fc, clean_fc)
            return {"status": "No repairs needed", "clean_fc": clean_fc}

        # Get list of problematic feature IDs
        problematic_oids = []
        issue_details = {}

        print(f"📝 Geometry issues found:")
        with arcpy.da.SearchCursor(validation_table, ["FEATURE_ID", "PROBLEM"]) as cursor:
            for row in cursor:
                feature_id, problem = row
                problematic_oids.append(feature_id)
                issue_details[feature_id] = problem
                print(f"  • Feature {feature_id}: {problem}")

        print(f"\n📊 {len(problematic_oids)} unique problematic features identified")

    except Exception as e:
        print(f"❌ Check Geometry failed: {e}")
        return {"error": f"Check Geometry failed: {e}"}

    print("\n" + "="*60)
    print("📂 STEP 2: EXTRACT BROKEN GEOMETRIES")
    print("="*60)

    # Extract the broken geometries to a feature class for inspection
    try:
        print("📋 Extracting broken geometries to feature class...")

        if len(problematic_oids) > 0:
            # Create WHERE clause to include only problematic records
            oid_field = arcpy.Describe(input_fc).OIDFieldName
            problematic_list = ",".join(map(str, problematic_oids))
            where_clause = f"{oid_field} IN ({problematic_list})"

            # Create feature class with broken geometries
            arcpy.Select_analysis(input_fc, broken_geometries_fc, where_clause)

            # Add fields to track the geometry issues
            arcpy.AddField_management(broken_geometries_fc, "GEOM_ISSUE", "TEXT", field_length=255)
            arcpy.AddField_management(broken_geometries_fc, "ISSUE_CODE", "TEXT", field_length=50)
            arcpy.AddField_management(broken_geometries_fc, "REPAIR_STATUS", "TEXT", field_length=50)

            # Update the geometry issue information
            with arcpy.da.UpdateCursor(broken_geometries_fc, ["OID@", "GEOM_ISSUE", "ISSUE_CODE", "REPAIR_STATUS"]) as cursor:
                for row in cursor:
                    oid = row[0]
                    if oid in issue_details:
                        issue_desc = issue_details[oid]
                        row[1] = issue_desc

                        # Extract issue code
                        if "(-148)" in issue_desc:
                            row[2] = "INSUFFICIENT_POINTS"
                        elif "(-155)" in issue_desc:
                            row[2] = "SELF_INTERSECTING"
                        elif "(-151)" in issue_desc:
                            row[2] = "SHELL_DONUT_ERROR"
                        else:
                            row[2] = "OTHER"

                        row[3] = "PENDING"
                        cursor.updateRow(row)

            broken_count = int(arcpy.GetCount_management(broken_geometries_fc).getOutput(0))
            print(f"✅ Broken geometries extracted: {broken_count} records")
            print(f"📂 Broken geometries saved to: {broken_geometries_fc}")

            # Show breakdown by issue type
            issue_type_counts = {}
            with arcpy.da.SearchCursor(broken_geometries_fc, ["ISSUE_CODE"]) as cursor:
                for row in cursor:
                    issue_code = row[0]
                    issue_type_counts[issue_code] = issue_type_counts.get(issue_code, 0) + 1

            print(f"📊 Issue type breakdown:")
            for issue_type, count in issue_type_counts.items():
                print(f"  • {issue_type}: {count} records")
        else:
            print("✅ No broken geometries to extract")
            broken_count = 0

    except Exception as e:
        print(f"❌ Failed to extract broken geometries: {e}")
        broken_count = 0

    print("\n" + "="*60)
    print("🧹 STEP 3: CREATE CLEAN DATASET (EXCLUDE PROBLEMATIC RECORDS)")
    print("="*60)

    # Create a clean dataset by excluding problematic records
    try:
        print("📋 Creating clean dataset by excluding problematic records...")

        # Create WHERE clause to exclude problematic records
        oid_field = arcpy.Describe(input_fc).OIDFieldName

        if len(problematic_oids) > 0:
            problematic_list = ",".join(map(str, problematic_oids))
            where_clause = f"{oid_field} NOT IN ({problematic_list})"
        else:
            where_clause = None

        # Create clean dataset
        if where_clause:
            arcpy.Select_analysis(input_fc, clean_fc, where_clause)
        else:
            arcpy.CopyFeatures_management(input_fc, clean_fc)

        clean_count = int(arcpy.GetCount_management(clean_fc).getOutput(0))
        print(f"✅ Clean dataset created: {clean_count:,} records")
        print(f"📊 Excluded {len(problematic_oids)} problematic records")

    except Exception as e:
        print(f"❌ Failed to create clean dataset: {e}")
        return {"error": f"Failed to create clean dataset: {e}"}

    print("\n" + "="*60)
    print("🛠️ STEP 4: ATTEMPT TO REPAIR PROBLEMATIC RECORDS")
    print("="*60)

    # Try to repair the problematic records individually
    repaired_count = 0
    unrepairable_count = 0

    # Create feature class for successfully repaired records
    try:
        arcpy.CreateFeatureclass_management(
            output_gdb, "RepairedRecords", "POLYGON",
            template=input_fc, spatial_reference=spatial_ref
        )
        repaired_records_fc = os.path.join(output_gdb, "RepairedRecords")

        # Create table for unrepairable records
        arcpy.CreateTable_management(output_gdb, "UnrepairableRecords")
        unrepairable_table = os.path.join(output_gdb, "UnrepairableRecords")
        arcpy.AddField_management(unrepairable_table, "FEATURE_ID", "LONG")
        arcpy.AddField_management(unrepairable_table, "PROBLEM", "TEXT", field_length=255)
        arcpy.AddField_management(unrepairable_table, "TREEFARM_ID", "LONG")
        arcpy.AddField_management(unrepairable_table, "PARCELNUMBER", "SHORT")

    except Exception as e:
        print(f"❌ Failed to create repair structures: {e}")
        return {"error": f"Failed to create repair structures: {e}"}

    # Process each problematic record
    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["FeatureID", "TreeFarmID", "ParcelNumber", "Problem", "RepairAttempt", "Status", "Timestamp"])

        with arcpy.da.InsertCursor(repaired_records_fc, ["SHAPE@", "treefarm_id", "parcelnumber"]) as repaired_writer, \
             arcpy.da.InsertCursor(unrepairable_table, ["FEATURE_ID", "PROBLEM", "TREEFARM_ID", "PARCELNUMBER"]) as unrepairable_writer:

            for feature_id in problematic_oids:
                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                problem = issue_details[feature_id]

                print(f"🔧 Attempting to repair Feature {feature_id}: {problem}")

                # Try to access the problematic record
                try:
                    where_clause = f"{oid_field} = {feature_id}"

                    with arcpy.da.SearchCursor(input_fc, ["SHAPE@", "treefarm_id", "parcelnumber"], where_clause) as cursor:
                        row = next(cursor, None)
                        if row:
                            shape, treefarm_id, parcelnumber = row

                            if shape is None:
                                print(f"  ❌ Null geometry - cannot repair")
                                unrepairable_writer.insertRow([feature_id, problem, treefarm_id, parcelnumber])
                                logwriter.writerow([feature_id, treefarm_id, parcelnumber, problem, "Null geometry", "Unrepairable", current_time])
                                unrepairable_count += 1
                                continue

                            # Try different repair methods
                            repair_methods = [
                                ("buffer_zero", lambda s: s.buffer(0)),
                                ("buffer_small", lambda s: s.buffer(0.01).buffer(-0.01)),
                                ("densify_buffer", lambda s: s.densify("DISTANCE", 10).buffer(0)),
                                ("generalize_buffer", lambda s: s.generalize("POINT_REMOVE", 1).buffer(0)),
                            ]

                            repaired = False
                            for method_name, repair_func in repair_methods:
                                try:
                                    repaired_shape = repair_func(shape)
                                    if repaired_shape and not repaired_shape.isEmpty:
                                        # Test if repair worked by checking it against Check Geometry
                                        repaired_writer.insertRow([repaired_shape, treefarm_id, parcelnumber])
                                        repaired_count += 1
                                        repaired = True
                                        print(f"  ✅ Repaired using {method_name}")
                                        logwriter.writerow([feature_id, treefarm_id, parcelnumber, problem, method_name, "Repaired", current_time])
                                        break
                                except Exception as repair_err:
                                    continue

                            if not repaired:
                                print(f"  ❌ All repair methods failed")
                                unrepairable_writer.insertRow([feature_id, problem, treefarm_id, parcelnumber])
                                logwriter.writerow([feature_id, treefarm_id, parcelnumber, problem, "All methods failed", "Unrepairable", current_time])
                                unrepairable_count += 1
                            else:
                                # Update the broken geometries feature class with repair status
                                try:
                                    with arcpy.da.UpdateCursor(broken_geometries_fc, ["OID@", "REPAIR_STATUS"], f"OID@ = {feature_id}") as update_cursor:
                                        for update_row in update_cursor:
                                            update_row[1] = "REPAIRED"
                                            update_cursor.updateRow(update_row)
                                except:
                                    pass
                        else:
                            print(f"  ❌ Record not found")
                            unrepairable_count += 1

                except Exception as access_err:
                    print(f"  ❌ Cannot access record: {access_err}")
                    unrepairable_writer.insertRow([feature_id, problem, None, None])
                    logwriter.writerow([feature_id, "Unknown", "Unknown", problem, "Access failed", "Unrepairable", current_time])
                    unrepairable_count += 1

                    # Update broken geometries status
                    try:
                        with arcpy.da.UpdateCursor(broken_geometries_fc, ["OID@", "REPAIR_STATUS"], f"OID@ = {feature_id}") as update_cursor:
                            for update_row in update_cursor:
                                update_row[1] = "UNREPAIRABLE"
                                update_cursor.updateRow(update_row)
                    except:
                        pass

    print(f"\n📊 Repair Results:")
    print(f"  ✅ Successfully repaired: {repaired_count}")
    print(f"  ❌ Unrepairable: {unrepairable_count}")

    print("\n" + "="*60)
    print("🎯 STEP 5: CREATE FINAL DATASET")
    print("="*60)

    # Merge clean dataset with repaired records
    try:
        print("📋 Creating final dataset (clean + repaired records)...")

        # Copy clean dataset to final output
        arcpy.CopyFeatures_management(clean_fc, repaired_fc)

        # Append repaired records if any
        if repaired_count > 0:
            arcpy.Append_management(repaired_records_fc, repaired_fc, "NO_TEST")

        final_count = int(arcpy.GetCount_management(repaired_fc).getOutput(0))
        print(f"✅ Final dataset created: {final_count:,} records")

    except Exception as e:
        print(f"❌ Failed to create final dataset: {e}")
        return {"error": f"Failed to create final dataset: {e}"}

    # Update final broken geometries status for any remaining unrepairable records
    try:
        with arcpy.da.UpdateCursor(broken_geometries_fc, ["REPAIR_STATUS"]) as cursor:
            for row in cursor:
                if row[0] == "PENDING":
                    row[0] = "UNREPAIRABLE"
                    cursor.updateRow(row)
    except:
        pass

    # Final summary
    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n✅ Targeted geometry fix completed in {total_time/60:.1f} minutes")

    summary = {
        "📊 Total Original Records": total_records,
        "🔍 Actual Geometry Problems": actual_issues,
        "📂 Broken Geometries Extracted": broken_count,
        "🧹 Clean Records": clean_count,
        "✅ Successfully Repaired": repaired_count,
        "❌ Unrepairable Records": unrepairable_count,
        "📊 Final Dataset Records": final_count,
        "📈 Dataset Quality": f"{((final_count/total_records)*100):.1f}% usable",
        "⏱️ Processing Time": f"{total_time/60:.1f} minutes",
        "🎯 Final Dataset": repaired_fc,
        "🧹 Clean Dataset": clean_fc,
        "📂 Broken Geometries": broken_geometries_fc,
        "🔍 Validation Results": validation_table,
        "📝 Repair Log": log_csv
    }

    print("\n" + "="*60)
    print("📋 TARGETED REPAIR SUMMARY")
    print("="*60)

    for k, v in summary.items():
        print(f"  {k}: {v}")

    print(f"\n🎉 SUCCESS! Your dataset is now ready for production use!")
    print(f"📂 Use this dataset: {repaired_fc}")
    print(f"📊 Quality: {((final_count/total_records)*100):.1f}% of original records are usable")
    print(f"🔍 Broken geometries for inspection: {broken_geometries_fc}")
    print(f"    • View in ArcGIS to see exactly which polygons had issues")
    print(f"    • Check GEOM_ISSUE field for problem descriptions")
    print(f"    • Check REPAIR_STATUS field to see repair results")

    return summary

# Main execution
if __name__ == "__main__":
    try:
        result = targeted_geometry_fix(
            input_fc=r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.DBO.TreeFarm",

            output_gdb=r"C:\temp\TreeFarm_TargetedFix.gdb"
        )

        print(f"\n🎉 MISSION ACCOMPLISHED!")
        print(f"You now have a properly repaired TreeFarm dataset! 🌲")

    except Exception as main_err:
        print(f"❌ Script failed: {main_err}")
        import traceback
        traceback.print_exc()

🚀 Starting targeted geometry fix at 2025-07-10 08:46:40
🎯 Focusing on the ~20 actual geometry problems (not 11,899!)
📊 Total records: 12,189
🗺️ Spatial Reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
📁 Creating output geodatabase: C:\temp\TreeFarm_TargetedFix.gdb

🔍 STEP 1: IDENTIFY ACTUAL GEOMETRY PROBLEMS
📋 Running ArcGIS Check Geometry (authoritative validation)...
📊 Actual geometry issues found: 20
📝 Geometry issues found:
  • Feature 681161: Shape has a structural error! (-151) A polygon outer shell does not completely enclose all donuts for the part
  • Feature 681171: Failed to select shape! (-148) The number of points is less than required for feature
  • Feature 681201: Shape has a structural error! (-155) Linestring or poly boundary is self-intersecting
  • Feature 681233: Shape has a structural error! (-155) Linestring or poly boundary is self-intersecting
  • Feature 681240: Shape has a structural error! (-151) A polygon outer shell does not completely enclose all donuts 